In [ ]:
############## 评分卡，就是还钱比较容易，信息度高
# ABCF卡有四张卡，A卡用来评估新用户的信用评估，申请者评估模型
# 获取数据，数据清洗，特征工程，模型开发，模型的检验与评估，不好就重新获取数据
# 好就模型上线，，检测与报告，再生成数据
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
#通常是再使用过程中一点点的导入各种库

#一定要和给数据的同事沟通好，问清楚给的数据是有什么特征什么意思,第一列当作索引
data = pd.read_csv("/home/aistudio/data/data15426/cs-training.csv", index_col=0)

# 探索数据,观察数据类型
data.head()

# 观察数据结构
data.shape
data.info()

# 去除重复值，当很多不可能重复的特征都全都重复了，那就删除了，银行数据的本身的问题
data.drop_duplicates(inplace=True)
data.info()


In [ ]:
# 删除后一定要恢复索引
data.index = range(data.shape[0])
#data.info()
#### 填补缺失值,isnull()返回布尔数组
#data.isnull().sum() / data.shape[0]
data.isnull().mean()
# 对家属人数进行操作
data["NumberOfDependents"].fillna(data["NumberOfDependents"].mean(), inplace=True) 
data.info()
# 对收入进行操作
# 收入栏缺失的人更可能是收入不稳定或者收入较低的人群，所以可以用四分位数来填充
# 也有可能是银行的失误，因此要与工作人员进行沟通，这里我们采取随机森林的方法进行填充


In [ ]:
# 填补函数
def fill_missing_rf(X, y, to_fill):
    '''
    使用随机森林进行填补
    参数：
    X：要填补的特征矩阵
    y: 完整的，没有缺失值的标签
    to_fill：字符串，要填补的缺失值的名称
    '''
    # 构建我们的新特征矩阵和新标签
    df = X.copy()
    fill = df.loc[:, to_fill]
    # 取其他列
    df = pd.concat([df.loc[:, df.columns != to_fill], pd.DataFrame(y)], axis=1)

    # 找出我们的训练集和测试集
    y_train = fill[fill.notnull()]
    y_valid = fill[fill.isnull()]
    x_train = df.iloc[y_train.index, :]
    x_valid = df.iloc[y_valid.index, :]

    # 用随机森林来填补缺失值
    from sklearn.ensemble import RandomForestRegressor as rfr
    rfr = rfr(n_estimators=100).fit(x_train, y_train)
    y_predict = rfr.predict(x_valid)

    return y_predict

In [ ]:
#data.head()
X = data.iloc[:, 1:]
y = data["SeriousDlqin2yrs"] # y = data.iloc[:, 0]
X.shape

In [ ]:
y_pred = fill_missing_rf(X, y, "MonthlyIncome")


In [ ]:
y_pred.shape
data.loc[:, "MonthlyIncome"].isnull().sum() # 0

In [ ]:
# 确认我们返回的数据是合理的之后，我们就可以将数据覆盖了
data.loc[data.loc[:, "MonthlyIncome"].isnull(), "MonthlyIncome"] = y_pred
# 我手里的数据集是完整的，所以运行时会报错，理解万岁哈哈哈

In [ ]:
# 描述性统计处理异常值
# 比如某一个商品的购买量超高，这时候就要去关注
# 处理日常值需要用箱线图或者3€法则。这里我们用普通的描述性统计
data.describe([0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])#返回不同样本数据量下的描述性统计值
# 结果看看最大值和最小值有没有异常值，这里可以看到收入是一个严重偏重的数据
# 年龄出现了0， 是奇怪的事情，这时候去问一下业务人员，看看多不多
(data["age"] == 0).sum() # 只有一个,说明可能有问题，直接删除
data = data[data["age"] != 0]
# 中间有个人违约两个月了98次。。。这个不合理,不正常的用户
data[data.loc[:, "NumberOfTimes90DaysLate"] > 90].count()
# 都违约在90天以上了不可能还是正常用户了,有225个人预期90天以上90次以上
data.loc[:, "NumberOfTimes90DaysLate"].value_counts()
# 确实有的离谱, 违约98 96次这种, 直接删除
data = data[data.loc[:, "NumberOfTimes90DaysLate"] < 90]
# 恢复索引，切记
data.index = range(data.shape[0])


In [ ]:
# 出现的偏态数据时
# 正态分布收敛下降的更快，但为什么不做标准化呢，也不统一量纲
# 我们是为了业务服务的，现在的评分卡是给业务人员服务的，当我们标准化之后，数据的大小和范围都会改变，比如
# 很难理解年龄在0.004-0.005之间的含义是什么。新用户填写数据天生就是数据量纲不统一的，我们的确可以将所有的数据导入后进行标准化
# 然后导入算法计算，但是最后给到业务人员说中的数据变成了一堆优美的统计数据，但根本看不懂。所以尽量保持数据的原貌，即使量纲不一


In [ ]:
# 处理不均衡数据
import imblearn
# 这个库是专门处理不平衡数据的库，里面也是一个个类，用法和sklearn类似
from imblearn.over_sampling import SMOTE
# over_sampling 上采样

sm = SMOTE(random_state=42)# 实例化
X, y = sm.fit_sample(X, y) # 已经平衡后的样本,返回已经平衡后的数据和标签
n_sample_ = X.shape[0]
x = data.iloc[:, 1:]
y = data.iloc[:, 0]

y.value_count()#分别显示0 1有多少

n_sample = x.shape[0]# 样本量
n_1_sample = y.value_count()[1]# 1的样本量
n_0_sample = y.value_count()[0]# 0的样本量
print("样本个数：{}；1占{:.2%};0占{:.2%}".format(n_sample, n_1_sample / n_sample, n_0_sample / n_sample))
# 可以看出违约的人不多，很多人是忘记了还钱日期，有的人是真的还款困难，只要有钱就一定会换上。真正需要抓的是恶意违约的人，但
# 人数很少，样本不均衡，这就是银行建模的痛点，找到少数类
# 逻辑回归中，最多使用上采样来平衡样本、
pd.Series(y).value_counts()

In [ ]:
### 开始进行训练数据的划分
from sklearn.model_selection import train_test_split
X = pd.DataFrame(X)
y = pd.DataFrame(y)
x_train, x_valid, y_train, y_valid = train_test_split(X, y, random_state=420)
model_data = pd.concat([y_train, x_train],axis=1)
model_data# 索引都乱了
model_data.index = range(model_data.shape[0])
data.columns#  指代索引
model_data.columns = data.columns

vali_data = pd.concat([y_valid, x_valid], axis=1)
vali_data.index = range(vali_data.shape[0])
vali_data.columns = data.columns

In [ ]:
model_data.to_csv("~/xunlianlujing.csv")
vali_data.to_csv("~/ceshilujing.csv")

In [ ]:
#################模型建立流程，分箱
# 按照等频对需要分享的列进行分箱
# “age”例子
#dataframe['列名']当这个类存在的时候，就是索引，当这个列名不存在的时候，dataframe会自动生成一个叫这个列名的列
model_data['qcut'], updown = pd.qcut(model_data['age'], retbins=True, q=20)
'''
pd.qcut基于分位数的分箱函数，本质是将连续性的变量离散化，只能够处理一维数据，返回箱子的上限和下限
参数q 要分箱的个数
retbins=true同时返回结构为索引为样本索引，元素为分到的箱子的series，现在返回两个值，每个样本属于哪个箱子，以及所有箱子的上限和下限
'''
#在这里让model——data新加一列叫做分箱，这一列其实就是每个样本对应的箱子，索引为样本索引，元素为分到的箱子的series
model_data['qcut']
#所有箱子的上限和下限
updown

In [ ]:
model_data['qcut'].value_counts()
updown#直接返回分箱上下限

In [ ]:
# 统计每个分箱0和1的个数
# 这里使用了数据透视表的功能groupby
count_y0 = model_data[model_data["SeriousDlqin2yrs"] == 0].groupby(by="qcut").count()["SeriousDlqin2yrs"]
count_y1 = model_data[model_data["SeriousDlqin2yrs"] == 1].groupby(by="qcut").count()["SeriousDlqin2yrs"]
#model_data[model_data["seriousdlqin2yrs"] == 0]取出所有标签为0的样本
#之后按照qcut进行聚合，count进行数这个组下面有多少条目
# 再[]取出相应的列

# num_bins值分别为每个区间的上界和下界，0出现的次数，1出现的次数
# zip列表变元组[*]打开惰性对象，可以查看里面的值
# updown,updown[1:]生成下限和上限
num_bins = [*zip(updown, updown[1:], count_y0, count_y1)]

# 注意zip会按照最短的那一个列表来进行结合
num_bins




In [ ]:
############## 确保每个箱子中都有0和1
for i in range(20):
    if 0 in num_bins[0][2:]:
        num_bins[0:2] = [(
            num_bins[0][0],
            num_bins[1][1],
            num_bins[0][2]+num_bins[1][2],
            num_bins[0][3]+num_bins[1][3])]
        continue
'''
如果原本的第一组和第二组都没有包含正样本或者负样本，那即使合并之后，第一行的组还是没有包含两种样本
所以我们每次结束后都检查，第一组是否已经包括了两种样本
continue跳出本次循环，如果第一组中依旧没有两种样本，则再次循环，继续合并，每合并一次就循环检查一次，如果
第一组中已经有两种样本，则进行下面的代码
'''
# 已经确认第一组中肯定包括两种样本了，如果其他组没有包含就像前合并
# 此时num_bins已经被上面的代码处理过了，可能被合并过也可能没有
# 但无论如何，我们都要再num_bins中进行遍历
    for i in range(len(num_bins)):
        if 0 in num_bins[i][2:]:
            num_bins[i-1:i+1] = [(
                num_bins[i-1][0],
                num_bins[i][1],
                num_bins[i-1][2]+num_bins[i][2],
                num_bins[i-1][3]+num_bins[i][3]
            )]
            break
        # 如果对第一组和对后面所有组的判断中，都没有进入if去合并，则提前结束所有的循环
    else:
        break
'''
倒数第二个break只有再进入if才会被执行
也就是说，只有发生了合并，才会打断for i in range(len())这个循环
为什么打断这个循环呢，因为我们是在range(len)中去遍历
但合并发生后，len(num——bins)发生了变化，但循环却不会重新开始
举个例子，本来是5组，for i in range(len(5))是第一次循环，所一循环一直时01234跑五次
合并后numbins只剩下4组了，索引不在了，循环报错
因此一旦合并，我们就break跳出循环
循环会回到for i in range(20)里面
此时判断第一组是否有两种标签的代码不会被触发，但for i in range(len(num_bins))会被重新执行，这样就
更新了i的取值，循环就不会报错哦了
'''


In [ ]:
for i in range(5):
    print(i)
else:
    print(i+1)
# for else
# while else
# 只要不报错都会执行else

In [ ]:
# 定义woe和IV函数
import numpy as np
def get_woe(num_bins):
    # 通过num_bins计算woe
    columns = ["min", "max", "count_0", "count_1"]
    df = pd.DataFrame(num_bins, columns=columns)
    df["total"] = df.count_0 + df.count_1
    df["percentage"] = df.total / df.total.sum()
    df["bad_rate"] = df.count_1 / df.total
    df["good%"] = df.count_0 / df.count_0.sum()
    df["bad%"] = df.count_1 / df.count_0.sum()
    df["woe"] = np.log(df['good%'] / df["bad%"])
    return df
def get_iv(bins_df):
    rate = bins_df["good%"] - bins_df["bad%"]
    iv = np.sum(rate * bins_df.woe)
    return iv


In [ ]:
# 进行卡方检验相邻箱子
# 所有箱子不同都因为count0和count1的不同，所以只用检验这两列

import matplotlib.pyplot as plt
import scipy

In [ ]:
x1 = num_bins[0][2:]
x2 = num_bins[1][2:]
# 提取后面两个数

In [ ]:
scipy.stats.chi2_contingency([x1, x2])
# 返回一个元组，第一个时卡方值，第二个是pvalue
# 另所有组的pvalue
axisx = []
IV = []
while len(num_bins) > 2:
    pvs = []
    # 获取num_bins两两之间的卡方检验的置信度
    for i in range(len(num_bins)-1):
        x1 = num_bins[i][2:]
        x2 = num_bins[i+1][2:]
        pv = scipy.stats.chi2_contingency([x1, x2])[1]
        pvs.append(pv)
    max(pvs)
    pvs.index(max(pvs))
    # 在哪个组里出现最大pvalue

    # 之后进行合并
    i = pvs.index(max(pvs))
    # 合并成一行
    #a = [1, 2, 3, 4, 5]
    #a[1:3] = [2]
    #a只剩四个了
    num_bins[i: i+2] = [(
        num_bins[i][0],
        num_bins[i+1][1],
        num_bins[i][2]+num_bins[i+1][2],
        num_bins[i][3]+num_bins[i+1][3]
    )]
    bins_df = get_woe(num_bins)
    axisx.append(len(num_bins))
    IV.append(get_iv(bins_df))

# 每次外层的while循环都会得到新的箱子的数量
plt.figure()
plt.plot(axisx,IV)
plt.xticks()
plt.show()
# 找到转折点
#将以上函数变成自动分箱的函数

In [ ]:
#为最优分箱画图，就把上面所有的代码都合并起来
# 可以画出来每一个特征的iv曲线，然后再进行分析
#找到陡峭的哪个箱子数
auto_col_bins = {
    "aa":3,
    "bb":4,
    "cc":5
}
# 中间有的特征画不出图画因为原来就几个离散型的数据，分不出20个箱子
# 对不能自动分箱的函数进行手动自动分箱
hand_bins = {
    "a":[1,2,3,4],
    "b":[2, 3,4,5],
    "c": [4, 5,6, 7]
}
# 就是有ab特征然后有几个离散的数据
# 保证区间覆盖使用np.inf替换最大值，-np.inf替换最小值
hand_bins = {k:[-np.inf, *v[:-1],np.inf] for k, v in hand_bins.items()}
hand_bins

In [ ]:
bins_of_col = {}
# 生成自动分箱的分箱区间和分箱后的IV值
for col in auto_col_bins:
    # 这个函数是自定义的，找到最优的分箱情况，并返回IV值
    bins_df = graphforbestbin(model_data,col,"seriousflqin2yrs"
                                            ,n=auto_col_bins[col]# 使用字典的性质来取出每个特征所对应的箱子的个数
                                            ,q=20
                                            ,graph=False)
    # union对集合取并集
    bins_list = sorted(set(bins_df["min"]).union(bins_df["max"]))
    # 保证区间覆盖使用np.inf替换最大值
    bins_list[0], bins_list[-1] = -np.inf, np.inf
    bins_of_col[col] = bins_list

# 合并手动分箱数据
# update将一个字典添加到一个字典当中
bins_of_col.update(hand_bins)
bins_of_col#返回字典，特征的名称对应的分箱上下界


In [ ]:
#以上已经把箱子都分出来了
# 这时再自己计算woe计算出来，再计算出原始数据，woe代表每个箱子上不违约的概率
# 利用woe替换箱子,把woe映射到model_data上,最后建模利用woe而不是箱子，箱子是个区间不好计算
data = model_data.copy()
# 函数pd.cut,可以根据已知的分箱间隔把数据分箱
# 参数为pd.cut(数据， 以列表表示的分箱间隔)
data = data[["age", "seriouosdlqin2yrs"]].copy()
data["cut"] = pd.cut(data['age'],[-np.ing, 48, 58, 64, 74, np.inf])
data

# 将数据按分享结果聚合，并取出其中的标签值.取出每个箱子里面的0 1数量
data.groupby('cut')["seriousflqin2yrs"].value_counts()
# 使用unstack()()来讲树形结构变成表状结构，就是把0 1 变成两列
data.groupby('cut')["seriousflqin2yrs"].value_counts().unstack()

bins_df = data.groupby('cut')["seriousflqin2yrs"].value_counts().unstack()
#bins_df["woe"] = np.log((bins_df[0]/bins_df[0].sum()) / bins_df[1] / bins_df[1].sum())
bins_df["woe"] = np.log(bins_df["good%"] / bins_df["bad%"])

# 把上面的步骤包装成函数
def get_woe(df, col, y, bins):
    df = df[[col, y]].copy()
    df["cut"] = pd.cut(df[col], bins)
    bins_df = df.groupby("cut")[y].value_counts().unstack()
    woe = bins_df["woe"] = np.log(bins_df["good%"] / bins_df["bad%"])
    return woe


In [ ]:
# 将所有特征的woe存贮在字典中
woeall = {}
for col in bins_df_col:
    woeall[col] = get_woe(model_data, col, "seriousdlqin2yrs", bins_of_col[col])
# bins_of_col是字典，特征对应的箱子


In [ ]:
# 把woe映射到model_datazhong
# 不希望覆盖原始的数据，创建一个新的dataframe，索引和原始数据model-data一样
model_woe = pd.DataFrame(index=model_data.index)# 就是单独的1，2，3 ，4.。。
# 将原数据分享后，按照箱的结果把woe结构用map映射到数据中
# 代表每一个样本所在的每一个箱的woe值
model_woe["age"] = pd.cut(model_data["age"], bins_of_col["age"], map(woeall["age"])
#改进，对所有特征进行
for col in bins_of_col:
    model_woe[col] = pd.cut(model_data[col], bins_of_col[col], map(woeall[col])

# 将标签补充到数据中
model_woe["seriousdlqin23yrs"] = model_data["seriousdlqin23yrs"]

# 这就是我们的建模数据了
model_woe.head()

In [ ]:
##### 建模与模型验证
# 弄完了训练集，接下来是测试集，在已经有分箱的情况下，只要将已经计算好的woe映射到测试集中就好
# 处理测试集
valid_woe = pd.DataFrame(index=valid_data.index)
for col in bins_of_col:
    model_woe[col] = pd.cut(valud_data[col], bins_of_col[col], map(woeall[col])

valid_woe["seriousdlqin23yrs"] = valid_data["seriousdlqin23yrs"]
valid_x = valid_woe.iloc[:, :-1]
valid_y = valid_woe.iloc[:, -1]

In [ ]:
########开始建模
######少数类需要用recall来衡量
X = model_woe.iloc[:, :-1]
y = model_woe.iloc[:, -1]
valid_x = valid_woe.iloc[:, :-1]
valid_y = valid_woe.iloc[:, -1]
from sklearn.linear_model import LogisticRegression as LR
lr = LR().fit(X, y)
lr.score(valid_x, valid_y)
# 返回86.59%的精确性
#下面进行学习曲线

In [ ]:
c_1 = np.linspace(0.01, 1, 20)
c_2 = np.linspace(0.01, 0.2, 20)

score = []
for i in c_2:
    lr = LR(solver="libinear", C=i).fit(X, y)
    score.append(lr.score(valid_x, valid_y))
plt.figure()
plt.plot(c_1, score)
plt.show()
# 看迭代了多少次
lr.n_iter_
# 模型上限就是这个

# 模型最好就在86%精确率

# 可以换一种评价指标roc曲线,找出少数类的能力，是指判断对一个少数类判断错一个多数类的可能性
import scikitplot as skplt
valid_proba_df = pd.DataFrame(lr.predict_proba(valid_x))
skplt.metrics.plot_roc(valid_y, valid_proba_df, plot_micro=False,figsize=(6,6), plot_macro=False)
# 下面的是auc面积，面积越大越好
# 预测的还挺好

In [ ]:
# 制作评分卡
B = 20 / np.log(2)
A = 600 + B * np.log(1 / 60)
# 不受各因素影响的基础分，是将截距log（odds)带入就可以，而其他各个特征各个分档的分数，也是将系数带入进行计算
base_score = A - B * lr.intercept_
base_score

# woeall每个特征每个箱子对应的woe
score_age = woeall["age"] * (-B * lr.coef_[0][0])

score_age# age之下每个箱子的分数，来一个人就是通过箱子来打分

# lr.coef_建模后的系数[0][0]取出age对应的特征
#lr.coef_ * B 是b * log(odds)
# 通过循环，把所有的评分卡都写到本地文件
file = "lujing"
# 打开文件并且写入,若文件不存在就生成之后再打开
with open(file, "w") as fdata:
    fdata.write("base_score, {}\n".format(base_score))
# 在文件里写入基础分数
for i, col in enumerate(X.columns):
    score = woeall[col] * (-B * lr.coef_[0][i])
    # 列的名字是score
    score.name = "score"
    # 索引名字是箱的上下限
    score.index.name = col
    # 保留标题
    score.to_csv(file, header=True, mode="a")
# 分数分组写入，每个特征每个分箱的分数都显示出来


